<a href="https://colab.research.google.com/github/tluxxx/weekly-pattern-in-stock-markets/blob/main/weekly_patterns_(part_1_16_week_cycle_with_pattern_according_to_Gebert).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploiting weekly patterns in stock market for trading
#### part 1: testing a trading pattern in a 16 weeks cycle

This series is based on a strategy-idea originally generated by Thomas Gebert (Kurzfrist-Strategien fuer Anleger, Boersenbuchverlag, 2020 (in German)). The original strategy was tested and expanded. Further modifications are investigated. The this part contains:

*   implementing and testing of the original strategy during the years [2000-2018]
*   expanding of the original strategy to the years after publication of the original study [2019-2024]
*   modification of the original strategy by shifting the trading time
*   including realistic fees into the analyses
*   analyzing key statistical parameters
*   visualisation of the results
*   sensitivity analyses





#1. Preparation & Data Upload

In [1]:
# preparation
from google.colab import drive
drive.mount("/content/gdrive")
# adding the current colab-directory
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/weekly_pattern')

!pip install tabulate

Mounted at /content/gdrive


In [2]:
# importing all standard modules
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt
from tabulate import tabulate

In [3]:
# importing customized modules
from helpers_pattern_01 import *

In [22]:
# re-importing customized modules (if required)
import importlib
import helpers_pattern_01
importlib.reload(helpers_pattern_01)
from helpers_pattern_01 import *

In [4]:
# direct download price dataframe from yfinance
end_date = '2024-03-30'
start_date = '2000-01-01'
price = yf.download('^GDAXI', start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


# 2. Original Strategy by Gebert (slightly simplified)
### Features:  
*   16-weeks cycle
*   tradetime for opening positions (buy/short) & closing positions (sell/cover): each monday 17:30 (at close)

### Modifications:
*   Original Strategy: If first trading day of a week is not Monday, trading on the last trading day of the previous week
*   Proposed Simplification: if first trading day of a week is not Monday, trading on the first trading day of that particular week


In [5]:
# original gebert pattern and setting other parameters
pos_original = [0, 0, 0, 0, 0, 0, 0, -1, 0, 0, -1, 0, 1, 1, 1, -1]
shift_weeks = 12
pos = pos_original
# transforming daily into weekly data and calculting weekly trading positions
price_w = transforming_daily_weekly_close(price)
pos_weekly(price_w, pos, shift_weeks)

In [6]:
# check if 2018-11-12 is in week_type 4
price_w[price_w.index >'2018-11-01'].head(3)

,price,week_nb,week_type,pos
start_w,,,,
2018-11-05,11494.959961,983,3,0
2018-11-12,11325.440430,984,4,0
2018-11-19,11244.540039,985,5,0


In [7]:
# calculation and plottint/printing of pnl time-series and final results for the [2000-2018] period
price_w_original = price_w[price_w.index < '2018-12-31']
pnl_original = pnl_calculation(price_w_original)
title='Equity Curve for 16-week-cycle (orig.pattern) vs. Buy & Hold Instrument: DAX [2000-2018]'
pnl_plotting(pnl_original, title)
x1, x2, x3 = pnl_original['pnl_b&h'][-1], pnl_original['pnl_strategy'][-1], pnl_original['pnl_strategy2'][-1]
print(f' *****************************************************************************')
print(f'Summary of results:  Buy & Hold : {x1:.2f},   Strategy: {x2:.2f},   Strategy2:  {x3:.2f}')

 *****************************************************************************
Summary of results:  Buy & Hold : 1.54,   Strategy: 23.56,   Strategy2:  4.36


In [8]:
# calculation and plottint/printing of pnl time-series and final results for the [2000-2024] period
pnl_all = pnl_calculation(price_w)
title = 'Equity Curve for 16-week-cycle (orig.pattern) vs. Buy & Hold Instrument: DAX [2000-2024]'
pnl_plotting(pnl_all, title)
x1, x2, x3 = pnl_all['pnl_b&h'][-1], pnl_all['pnl_strategy'][-1], pnl_all['pnl_strategy2'][-1]
print(f' *****************************************************************************')
print(f'Summary of results:  Buy & Hold : {x1:.2f},   Strategy: {x2:.2f},   Strategy2:  {x3:.2f}')

 *****************************************************************************
Summary of results:  Buy & Hold : 2.71,   Strategy: 26.83,   Strategy2:  4.51


In [9]:
# calculation and plotting/printing of pnl time-series and final results for the [2019-2024] period
# (i.e. after the publication in 2019)
price_w_recent = price_w[price_w.index>'2019-01-01']
pnl_recent = pnl_calculation(price_w_recent)
title = 'Equity Curve for 16-week-cycle (orig.pattern) vs. Buy & Hold Instrument: DAX [2019-2024]'
pnl_plotting(pnl_recent, title)
x1, x2, x3 = pnl_recent['pnl_b&h'][-1], pnl_recent['pnl_strategy'][-1], pnl_recent['pnl_strategy2'][-1]
print(f' *****************************************************************************')
print(f'Summary of results:  Buy & Hold : {x1:.2f},   Strategy: {x2:.2f},   Strategy2:  {x3:.2f}')

 *****************************************************************************
Summary of results:  Buy & Hold : 1.73,   Strategy: 1.16,   Strategy2:  1.18


# 3. Modification of simplified Original Strategy

*   identical to the slightly modified Original Strategy of Gebert (except tradetime)
*   tradetime for opening positions (buy/short) & closing positions (sell/cover): each monday 9:00 (at open)



In [10]:
# transforming daily into weekly data and calculting weekly trading positions
price_w = transforming_daily_weekly_open(price)
pos_weekly(price_w, pos, shift_weeks)

In [11]:
# check if 2018-11-12 is in week_type 4
price_w[price_w.index >'2018-11-01'].head(3)

,price,week_nb,week_type,pos
start_w,,,,
2018-11-05,11522.330078,983,3,0
2018-11-12,11591.589844,984,4,0
2018-11-19,11384.000000,985,5,0


In [12]:
# calculation and plottint/printing of pnl time-series and final results for the [2000-2024] period
pnl_all = pnl_calculation(price_w)
title = 'Equity Curve for 16-week-cycle (orig.pattern) vs. Buy & Hold Instrument: DAX [2000-2024]'
pnl_plotting(pnl_all, title)
x1, x2, x3 = pnl_all['pnl_b&h'][-1], pnl_all['pnl_strategy'][-1], pnl_all['pnl_strategy2'][-1]
print(f' *****************************************************************************')
print(f'Summary of equity development: Buy & Hold : {x1:.2f},   Strategy: {x2:.2f},   Strategy2:  {x3:.2f}')

 *****************************************************************************
Summary of equity development: Buy & Hold : 2.62,   Strategy: 31.26,   Strategy2:  4.66


In [13]:
# calculation and plotting/printing of pnl time-series and final results for the [2019-2024] period
# (i.e. after the publication in 2019)
price_w_recent = price_w[price_w.index>'2019-01-01']
pnl_recent = pnl_calculation(price_w_recent)
title = 'Equity Curve for 16-week-cycle (orig.pattern) vs. Buy & Hold Instrument: DAX [2019-2024]'
pnl_plotting(pnl_recent, title)
x1, x2, x3 = pnl_recent['pnl_b&h'][-1], pnl_recent['pnl_strategy'][-1], pnl_recent['pnl_strategy2'][-1]
print(f' *****************************************************************************')
print(f'Summary of results:  Buy & Hold : {x1:.2f},   Strategy: {x2:.2f},   Strategy2:  {x3:.2f}')

 *****************************************************************************
Summary of results:  Buy & Hold : 1.74,   Strategy: 1.36,   Strategy2:  1.34


In [14]:
# export of results to excel file
with pd.ExcelWriter(r'/content/gdrive/MyDrive/Colab Notebooks/weekly_pattern/16_weeks_cycle (data).xlsx') as writer:
     price.to_excel(writer, sheet_name="daily data")
     price_w.to_excel(writer, sheet_name="weekly data")
     pnl_all.to_excel(writer, sheet_name="PnL")

# 4. Trade Statistics

In [23]:
# simulation of real conditions (incl. fee) and calculation of statistics
start_equity = 10000
fee = 0.0025                                                  # 0,25% of the traded volume
fixed_fee = 4.9                                               # 4,90 EUR pro trade
trades = trading_journal(price_w)                             # generation list of trades
trade_key_parameters(trades,price_w)                                  # key parameter independent from start_equity
trade_key_parameters2(trades, start_equity, fee, fixed_fee)   # key parameter depending on start_equity

********************************************************
start date:  2000-01-03
end date:    2024-03-25
┌───────────────────────────┬──────────────┬────────┬─────────┬────────┐
│  Parameter                │   All Trades │   LONG │   SHORT │   FLAT │
├───────────────────────────┼──────────────┼────────┼─────────┼────────┤
│ duration (days):          │     8,848.00 │        │         │        │
├───────────────────────────┼──────────────┼────────┼─────────┼────────┤
│ nb of trades:             │       317.00 │  80.00 │  237.00 │        │
├───────────────────────────┼──────────────┼────────┼─────────┼────────┤
│ duration of trades (%):   │       100.00 │  18.77 │   18.75 │  62.48 │
├───────────────────────────┼──────────────┼────────┼─────────┼────────┤
│ percent profitable (%):   │        58.36 │  71.25 │   54.01 │        │
├───────────────────────────┼──────────────┼────────┼─────────┼────────┤
│ profit factor:            │         2.42 │        │         │        │
├──────────────────

In [24]:
# real equity curves (w/o fee and incl fee)
eqs = real_equity_curve(price_w, start_equity, fee, fixed_fee)
fig = px.line(eqs, x=eqs.index, y=['value','value_wo_fees'])
title = 'Equity Curve for 16-week-cycle (orig.pattern) with and without fee. Instrument: DAX [2000-2024]'
fig.update_layout(title=title, xaxis_title='Date', yaxis_title='Equity (EUR)')
fig.update_layout(template = 'plotly_dark', autosize=False,width=1200,height=500)
fig.show()

end_value_wo_fee, end_value_fee =  eqs['value_wo_fees'][-1], eqs['value'][-1]
print(f' *****************************************************************************')
print(f' Equity development (in EUR):  Start Value {start_equity:,.1f}    End Value (w/o fees): {end_value_wo_fee:,.1f}   End Value incl. fees: {end_value_fee:,.1f}')

 *****************************************************************************
 Equity development (in EUR):  Start Value 10,000.0    End Value (w/o fees): 312,593.7   End Value incl. fees: 56,846.3


In [25]:
# distribution of trade-returns
fig = px.histogram(trades, x=['returns'], color='type', nbins= 20) #, marginal="rug", # can be `box`, `violin` hover_data=df.columns)
title = 'Distribution of Returns - 16 wks cycle,  modified strategy, Instrument: DAX [2000-2024]'
fig.update_layout(title=title, xaxis_title='Date', yaxis_title='# occurence')
fig.update_layout(template = 'plotly_dark', autosize=False,width=800,height=500)
fig.show()

In [26]:
# DrawDown of the Trades
uw = (pnl_all['pnl_strategy'] / pnl_all['pnl_strategy'].expanding().max()) - 1
fig = go.Figure()
fig.add_trace(go.Scatter(x=uw.index, y=uw, mode='none', fill='tozeroy', fillcolor='rgba(245, 227, 60, 0.6)', name='original pattern'))
title = 'Underwater-Plot 16-weeks-cycle (original pattern) DAX [2000-2024] '
fig.update_layout(title=title, xaxis_title='Date', yaxis_title='drawdown')
fig.update_layout(template = 'plotly_dark', autosize=False, width=1200, height=300)
fig.show()

In [27]:
# gantt-chart of trading positions of various patterns
various_trades = [trades]
labels = ['original pattern']
head_line = 'positions during the overall timeline (16-weeks-cycle), DAX [2000-2024]'
position_gantt(various_trades, labels, head_line)

#5. Sensitivity Analyses

Testing, how sensitive are the results regarding shifting the pattern within the 16 weeks cycle

In [28]:
# preparation of results dataframe, weekly data and setting parameters
result = pd.DataFrame(columns=['shift_wks','buyhold','strategy','strategy2'])
price_w2 = transforming_daily_weekly_open(price)
pos = pos_original
weeks = 16
for shift_weeks in range (weeks):
  # calculation of week-type and resulting positions depending on the shift
  pos_weekly(price_w2, pos, shift_weeks)
  pnl = pnl_calculation(price_w2)
  x1, x2, x3 = pnl['pnl_b&h'][-1], pnl['pnl_strategy'][-1], pnl['pnl_strategy2'][-1]
  result.loc[len(result.index)] = [shift_weeks, x1, x2, x3]

In [29]:
# plotting results
fig = px.bar(result, x=result['shift_wks'], y=result['strategy'])
fig.update_layout(title='PnL for 16 weeks cycle (various shift parameters)')
fig.add_hline(y=x1, line_dash='dot', line_color='green', annotation_text='Buy and Hold PnL', annotation_position='top left')
fig.update_layout(template = 'plotly_dark', autosize=False,width=800,height=400)
fig.update_layout(xaxis_title='shift parameter [in wks]', yaxis_title='PnL')